ToDo:
- [X] Pick more than one card
- [ ] Modify print statement based on Arcana
    - Major *doesn't need* "the" infront of it
        - Don't want "the The Tower"
    - Minor does 
        - "the Ten of Pentacles" 
- [ ] Add meaning? 
- [ ] Set random seed to 42
- [ ] Record - day, time, and item? Put them into a CSV? & Auto Export?
- [ ] Set up a Cron Job to randomly run?


In [1]:
import pandas as pd
import random
from math import comb

In [2]:
csv_file_path = '/Users/jebbonanno/Documents/workspace/tarot_card_reader/tarot_cards.csv'

In [3]:
tarot_df = pd.read_csv(csv_file_path)

In [4]:
word_to_number = {
    "zero": 0, "one": 1, "two": 2, "three": 3, "four": 4,
    "five": 5, "six": 6, "seven": 7, "eight": 8, "nine": 9
}

tarot_df['number_digit'] = tarot_df['Number'].apply(lambda x: word_to_number.get(x.lower(), None))
tarot_df['number_digit'].fillna(0, inplace=True)

In [5]:
tarot_df.sample(10).sort_index()

,Name,Suit,Number,Order,number_digit
9,The Hermit,Major Arcane,9,10,0.0
12,The Hanged Man,Major Arcane,12,13,0.0
37,Two of Cups,Cups,Two,37,2.0
52,Three of Swords,Swords,Three,52,3.0
56,Seven of Swords,Swords,Seven,56,7.0
57,Eight of Swords,Swords,Eight,57,8.0
65,Two of Pentacles,Pentacles,Two,65,2.0
66,Three of Pentacles,Pentacles,Three,66,3.0
72,Nine of Pentacles,Pentacles,Nine,72,9.0
77,King of Pentacles,Pentacles,King,78,0.0


In [6]:
#probabilities 

prob_major_arcana = comb(22, 3) / comb(78, 3)
print(f"Probality all Major Arcana: {round((prob_major_arcana*100),2)}%")

prob_minor_arcana = 4*comb(14, 3) / comb(78, 3)
print(f"Probability all Minor Arcana: {round((prob_minor_arcana*100),2)}%")

prob_royals = comb(16, 3) / comb(78 , 3) 
print(f"Probability all Royals (Page, Knight, Queen, King): {round((prob_royals*100),2)}%")


Probality all Major Arcana: 2.02%
Probability all Minor Arcana: 1.91%
Probability all Royals (Page, Knight, Queen, King): 0.74%


### Condition Check

In [7]:
def condition_check(three_card_pull):
    
    court_list = ['Page', 'Knight', 'Queen', 'King']
    
    if three_card_pull['Number'].nunique() == 1:
        print("Three of a Kind")
    elif three_card_pull['Suit'].nunique() == 1:
        print("A Three Card Flush")
    elif (three_card_pull['number_digit'][2] - three_card_pull['number_digit'][1] == 1 and 
          three_card_pull['number_digit'][1] - three_card_pull['number_digit'][0] == 1):
        print("A Straight")
    elif three_card_pull['Number'].isin(court_list).all() == True:
        print("A Court Pull")
    # else:
    #     print('Nothing Special')

### Three Card Pull

In [8]:
def three_card_pull_function():  # Renamed function to avoid conflict
    three_card_pull_df = tarot_df.sample(3).reset_index(drop=True)  # Renamed the variable

    for index, row in three_card_pull_df.iterrows():
        if index == 0:
            print(f"Past: {row['Name']}")
        elif index == 1:
            print(f"Present: {row['Name']}")
        elif index == 2:
            print(f"Future: {row['Name']}")
        else:
            print("Unexpected error")

    condition_check(three_card_pull_df)

In [9]:
for i in range(0,10):
    three_card_pull_function()
    print()

Past: Three of Pentacles
Present: Six of Cups
Future: King of Swords

Past: Eight of Swords
Present: Knight of Swords
Future: Six of Cups

Past: Ten of Swords
Present: Three of Cups
Future: The Hermit

Past: The Empress
Present: Five of Wands
Future: Eight of Swords

Past: Ace of Swords
Present: Four of Cups
Future: King of Cups

Past: Nine of Wands
Present: Knight of Swords
Future: The Tower

Past: The Fool
Present: King of Pentacles
Future: Page of Wands

Past: The High Priestest
Present: Six of Cups
Future: The Lovers

Past: Eight of Cups
Present: Seven of Pentacles
Future: Four of Cups

Past: Queen of Wands
Present: Ten of Pentacles
Future: Three of Pentacles

Past: Nine of Pentacles
Present: Page of Swords
Future: Seven of Wands

Past: The Emperor
Present: The Fool
Future: Page of Swords

Past: Seven of Pentacles
Present: Nine of Wands
Future: Queen of Wands

Past: Ace of Swords
Present: Temperance
Future: King of Pentacles

Past: Knight of Cups
Present: The World
Future: The Sun


### Single Pull

In [ ]:
def tarot_print_statement(card):
    if card[:3] == 'The':
        print(f"You drew {single_pick}.")
    else:
        print(f"You drew the {card}.")

In [ ]:
tarot_print_statement(tarot_df['Name'].sample(1).values[0])